<a href="https://colab.research.google.com/github/Yapping72/ICT3102-e-mc2-assignment-1/blob/main/ICT3102_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
corpus = [
    "The sky is blue.",
    "I love to read.",
    "Dogs are loyal pets.",
    "The sun sets in the west.",
    "Pizza is my favorite food.",
    "The ocean is vast and mysterious.",
    "Cats enjoy chasing mice.",
    "Basketball is a popular sport.",
    "Trees provide shade and oxygen.",
    "Music soothes the soul.",
    "Rainbows are beautiful after rain.",
    "I enjoy hiking in nature.",
    "Writing is a creative outlet.",
    "Learning is a lifelong journey.",
    "Flowers bloom in spring.",
    "The moon glows at night.",
    "Coffee helps me wake up.",
    "Travel broadens the mind.",
    "Science explores the unknown.",
    "Art inspires and uplifts.",
    "Silence can be powerful.",
    "Education is key to success.",
    "Happiness comes from within.",
    "Exercise is good for the body.",
    "Kindness matters in life.",
    "Family provides support and love.",
    "Laughter is the best medicine.",
    "Technology is constantly evolving.",
    "Success requires hard work.",
    "Adventure awaits those who seek it.",
    "Love conquers all.",
    "A smile brightens someone's day.",
    "Life is a journey, not a destination.",
    "Dream big and aim high.",
    "Patience is a virtue.",
    "The early bird catches the worm.",
    "Actions speak louder than words.",
    "Imagination knows no bounds.",
    "Courage is facing fear head-on.",
    "Hope keeps us going.",
    "Cherish every moment.",
    "Friendship is a precious gift.",
    "The world is full of beauty.",
    "Gratitude brings happiness.",
    "Change is inevitable, growth is optional."
]